## Note
Feeding angle directly to the model with ( * PI / 180 ) and without it makes no difference (cuz it's multiply with a constant)

In [1]:
from tqdm import tqdm
from ase.io import read, write
from ase import Atoms
# from rdkit.Chem import Draw
# from rdkit.Chem import AllChem as Chem
from collections import Counter

# import cairosvg
import pybel
import numpy as np
import pandas as pd
import os

ImportError: DLL load failed: The specified module could not be found.

## Util functions

In [94]:
PI = np.pi
def get_cell_parameters(cifname):
    content = open(cifname, 'r').readlines()
    mol = read(cifname)
    n_atoms = mol.get_global_number_of_atoms()
    mol_avg_mass = mol.get_masses().mean()
    
    # calculate charges (not including charges of neutral element)
    atom_count = Counter(mol.get_chemical_symbols())
    charges = calc_charges(atom_count, charge_dict)
    
    for line in content:
        if "_cell_length_a" in line:
            a = float(line.split()[1])
        if "_cell_length_b" in line:
            b = float(line.split()[1])
        if "_cell_length_c" in line:
            c = float(line.split()[1])
        if "_cell_angle_alpha" in line:
            alpha_r = float(line.split()[1]) * PI / 180.0
        if "_cell_angle_beta" in line:
            beta_r = float(line.split()[1]) * PI / 180.0
        if "_cell_angle_gamma" in line:
            gamma_r = float(line.split()[1]) * PI / 180.0

    return a, b, c, alpha_r, beta_r, gamma_r, n_atoms, mol_avg_mass, charges, avg_radius

In [68]:
# some files are converted to the wrong SMILES, so it cannot be saved as image
def smiles_img_from_cif(cifname, dest):
    cif = next(pybel.readfile("cif", cifname))
    smiles = cif.write(format="smi")
    smiles = smiles[:smiles.rfind('\t')]
    mol = Chem.MolFromSmiles(smiles)
    Draw.MolToFile(mol, dest)

In [2]:
def smiles_from_cif(cifname):
    cif = next(pybel.readfile("cif", cifname))
    smiles = cif.write(format="smi")
    smiles = smiles[:smiles.rfind('\t')]
    return smiles

In [69]:
def svg_png_from_cif(cifname, dest_svg, dest_png):
    cif = next(pybel.readfile("cif", cifname))
    svg = cif.write(format="svg")
    # remove the text and add the missing closed tag in the end
    svg = svg[:svg.rfind('<text')]+'</svg>\n' 
    with open(dest_svg, 'w') as file:
        file.write(svg)
    cairosvg.svg2png(file_obj=open(dest_svg), write_to=dest_png, output_width=300, output_height=300)

In [70]:
def count_atom(formula):
    count = {}
    number = ""
    atom = ""
    for i in range(len(formula)):
        if formula[i].isalpha():
            if atom != '':
                if formula[i].islower():
                    atom += formula[i]
                else:
                    if number == '':
                        number = '1'
                    count[atom] = int(number)
                    atom = formula[i]
                    number = ''
            else:
                atom += formula[i]
        else:
            number += formula[i]
            
    # get the last atom
    if number == '':
        number = '1'
    count[atom] = int(number)
    
    return count

In [71]:
def calc_charges(atom_count, charge_dict):
    total_charge = 0
    for k,v in atom_count.items():
        total_charge += (v * charge_dict[k])
    return total_charge

In [108]:
def calc_avg_radius(n_atoms, atom_count, radius_dict):
    total_radius = 0
    for k,v in atom_count.items():
        total_radius += (v * radius_dict[k])
    mean_radius = total_radius / n_atoms
    return mean_radius

In [16]:
def calc_atoms_vol(cifname, vol_dict):
    mol = read(cifname)
    # calculate charges (not including charges of neutral element)
    atom_count = Counter(mol.get_chemical_symbols())
    total_vol = 0
    for k,v in atom_count.items():
        total_vol += (v * vol_dict[k])
    return total_vol

In [6]:
def calc_atoms_area(cifname, area_dict):
    mol = read(cifname)
    # calculate charges (not including charges of neutral element)
    atom_count = Counter(mol.get_chemical_symbols())
    total_area = 0
    for k,v in atom_count.items():
        total_area += (v * area_dict[k])
    return total_area

In [59]:
def get_desc_xyz(xyzname):
    mol = next(pybel.readfile("xyz", xyzname))
    desc = mol.calcdesc()
    n_atoms = len(mol.atoms)
    return desc['HBA1'], desc['HBA2'], desc['HBD'], desc['MW'], n_atoms

In [71]:
def get_desc_cif(cifname):
    mol = next(pybel.readfile("cif", cifname))
    desc = mol.calcdesc()
    return desc['HBA1'], desc['HBA2'], desc['HBD'], desc['nF'], desc['logP'], desc['MR'], desc['TPSA']

# Smiles from CIF

## Train

In [8]:
len_train = len(os.listdir('mof_cif_train'))
len_train

68613

In [ ]:
train_smiles = []
for i in tqdm(range(948,20001)):
    name = f'mof_unit_{i+1}'
    smiles = smiles_from_cif(f'mof_cif_train/{name}.cif')
    train_smiles.append(smiles)


  1%|▌                                                                         | 133/19053 [3:15:06<8:44:44,  1.66s/it]


  1%|█                                                                       | 268/19053 [4:36:45<159:18:03, 30.53s/it]


  2%|█▌                                                                      | 406/19053 [5:16:46<119:42:17, 23.11s/it]


  3%|██                                                                       | 540/19053 [5:20:37<28:37:22,  5.57s/it]


  4%|██▌                                                                       | 674/19053 [5:31:22<5:09:54,  1.01s/it]


...

 20%|██████████████▋                                                         | 3899/19053 [17:49:53<3:18:48,  1.27it/s]


 21%|██████████████▍                                                     | 4034/19053 [18:40:32<3164:04:18, 758.42s/it]


 22%|███████████████▊                                                        | 4168/19053 [18:53:59<9:18:52,  2.25s/it]


 23%|████████████████▎                                                       | 4303/19053 [18:58:54<3:23:47,  1.21it/s]


 23%|████████████████▎                                                       | 4329/19053 [19:00:48<1:22:28,  2.98it/s]

---

# Images from CIF

## Train

In [8]:
len_train = len(os.listdir('mof_cif_train'))
len_train

68613

In [ ]:
for i in tqdm(range(len_train)):
    name = f'mof_unit_{i+1}'
    svg_png_from_cif(f'mof_cif_train/{name}.cif', dest_svg=f'mof_images/svg/train/{name}.svg',
                     dest_png=f'mof_images/png/train/{name}.png')


  1%|▌                                                                         | 133/19053 [3:15:06<8:44:44,  1.66s/it]


  1%|█                                                                       | 268/19053 [4:36:45<159:18:03, 30.53s/it]


  2%|█▌                                                                      | 406/19053 [5:16:46<119:42:17, 23.11s/it]


  3%|██                                                                       | 540/19053 [5:20:37<28:37:22,  5.57s/it]


  4%|██▌                                                                       | 674/19053 [5:31:22<5:09:54,  1.01s/it]


...

 20%|██████████████▋                                                         | 3899/19053 [17:49:53<3:18:48,  1.27it/s]


 21%|██████████████▍                                                     | 4034/19053 [18:40:32<3164:04:18, 758.42s/it]


 22%|███████████████▊                                                        | 4168/19053 [18:53:59<9:18:52,  2.25s/it]


 23%|████████████████▎                                                       | 4303/19053 [18:58:54<3:23:47,  1.21it/s]


 23%|████████████████▎                                                       | 4329/19053 [19:00:48<1:22:28,  2.98it/s]

## Pretest

In [9]:
len_pretest = len(os.listdir('mof_cif_pretest'))
len_pretest

68613

In [ ]:
for i in tqdm(range(len_pretest)):
    name = f'mof_unit_pretest_{i+1}'
    svg_png_from_cif(f'mof_cif_pretest/{name}.cif', dest_svg=f'mof_images/svg/pretest/{name}.svg',
                     dest_png=f'mof_images/png/pretest/{name}.png')

  0%|                                                                            | 63/68613 [03:05<18:00:12,  1.06it/s]

---

# Charges

In [97]:
import pandas as pd
import numpy as np
from collections import Counter

In [110]:
df = pd.read_csv('atom_mass_charge.csv')
df

,Element,Energy level,Mass(u),Charge,Radius(pm)
0,H,1,1.0080,1,53
1,He,1,4.0026,0,31
2,Li,2,6.9400,1,167
3,Be,2,9.0122,2,112
4,B,2,10.8100,3,87
...,...,...,...,...,...
113,Fi,7,289.0000,0,-
114,Mc,7,290.0000,0,-
115,Lv,7,293.0000,0,-
116,Ts,7,294.0000,0,-


In [111]:
df = df.set_index('Element', drop=True)
radius_df = df['Radius(pm)'].replace({'-':0}).astype(float)
charge_df = df['Charge']

In [112]:
charge_dict = charge_df.to_dict()
charge_dict

{'H': 1,
 'He': 0,
 'Li': 1,
 'Be': 2,
 'B': 3,
 'C': 4,
 'N': -3,
 'O': -2,
 'F': -1,
 'Ne': 0,
 'Na': 1,
 'Mg': 2,
 'Al': 3,
 'Si': 4,
 'P': -3,
 'S': -2,
 'Cl': -1,
 'Ar': 0,
 'K': 1,
 'Ca': 2,
 'Sc': 0,
 'Ti': 3,
 'V': 0,
 'Cr': 0,
 'Mn': 0,
 'Fe': 0,
 'Co': 0,
 'Ni': 0,
 'Cu': 0,
 'Zn': 0,
 'Ga': 3,
 'Ge': 4,
 'As': -3,
 'Se': -2,
 'Br': -1,
 'Kr': 0,
 'Rb': 1,
 'Sr': 2,
 'Y': 0,
 'Zr': 0,
 'Nb': 0,
 'Mo': 0,
 'Tc': 0,
 'Ru': 0,
 'Rh': 0,
 'Pd': 0,
 'Ag': 0,
 'Cd': 0,
 'In': 3,
 'Sn': 4,
 'Sb': -3,
 'Te': -2,
 'I': -1,
 'Xe': 0,
 'Cs': 1,
 'Ba': 2,
 'La': 0,
 'Ce': 0,
 'Pr': 0,
 'Nd': 0,
 'Pm': 0,
 'Sm': 0,
 'Eu': 0,
 'Gd': 0,
 'Tb': 0,
 'Dy': 0,
 'Ho': 0,
 'Er': 0,
 'Tm': 0,
 'Yb': 0,
 'Lu': 0,
 'Hf': 0,
 'Ta': 0,
 'W': 0,
 'Re': 0,
 'Os': 0,
 'Ir': 0,
 'Pt': 0,
 'Au': 0,
 'Hg': 0,
 'Pb': 4,
 'Bi': -3,
 'Po': -2,
 'At': -1,
 'Rn': 0,
 'Fr': 1,
 'Ra': 2,
 'Ac': 0,
 'Th': 0,
 'Pa': 0,
 'U': 0,
 'Np': 0,
 'Pu': 0,
 'Am': 0,
 'Cm': 0,
 'Bk': 0,
 'Cf': 0,
 'Es': 0,
 'Fm': 0,
 'Md': 0,

In [113]:
radius_dict = radius_df.to_dict()
radius_dict

{'H': 53.0,
 'He': 31.0,
 'Li': 167.0,
 'Be': 112.0,
 'B': 87.0,
 'C': 67.0,
 'N': 56.0,
 'O': 48.0,
 'F': 42.0,
 'Ne': 38.0,
 'Na': 190.0,
 'Mg': 145.0,
 'Al': 118.0,
 'Si': 111.0,
 'P': 98.0,
 'S': 88.0,
 'Cl': 79.0,
 'Ar': 71.0,
 'K': 243.0,
 'Ca': 194.0,
 'Sc': 184.0,
 'Ti': 156.0,
 'V': 171.0,
 'Cr': 166.0,
 'Mn': 161.0,
 'Fe': 156.0,
 'Co': 152.0,
 'Ni': 149.0,
 'Cu': 145.0,
 'Zn': 142.0,
 'Ga': 136.0,
 'Ge': 125.0,
 'As': 114.0,
 'Se': 103.0,
 'Br': 94.0,
 'Kr': 88.0,
 'Rb': 265.0,
 'Sr': 219.0,
 'Y': 212.0,
 'Zr': 206.0,
 'Nb': 198.0,
 'Mo': 190.0,
 'Tc': 183.0,
 'Ru': 178.0,
 'Rh': 173.0,
 'Pd': 169.0,
 'Ag': 165.0,
 'Cd': 161.0,
 'In': 156.0,
 'Sn': 145.0,
 'Sb': 133.0,
 'Te': 123.0,
 'I': 115.0,
 'Xe': 108.0,
 'Cs': 298.0,
 'Ba': 253.0,
 'La': 0.0,
 'Ce': 0.0,
 'Pr': 247.0,
 'Nd': 206.0,
 'Pm': 205.0,
 'Sm': 238.0,
 'Eu': 231.0,
 'Gd': 233.0,
 'Tb': 225.0,
 'Dy': 228.0,
 'Ho': 226.0,
 'Er': 226.0,
 'Tm': 222.0,
 'Yb': 222.0,
 'Lu': 217.0,
 'Hf': 208.0,
 'Ta': 200.0,
 'W': 19

In [121]:
test_count = count_atom('C67H46N16Ni4O')
test_count

{'C': 67, 'H': 46, 'N': 16, 'Ni': 4, 'O': 1}

In [118]:
calc_charges(test_count, charge_dict)

314

In [122]:
calc_avg_radius(67+46+16+4+1, test_count, radius_dict)

63.1865671641791

---

# Train

In [72]:
train_df = pd.read_csv('train.csv')
train_df

,MOFname,volume [A^3],weight [u],surface_area [m^2/g],void_fraction,void_volume [cm^3/g],functional_groups,metal_linker,organic_linker1,organic_linker2,topology,CO2/N2_selectivity,heat_adsorption_CO2_P0.15bar_T298K [kcal/mol],CO2_working_capacity [mL/g]
0,mof_unit_1,1116.667429,875.240600,0.00,0.07899,0.0607,COOH-OEt,3,4,11,pcu,22.864166,6.786041,105.284502
1,mof_unit_2,2769.503842,2211.697211,603.61,0.13794,0.1040,F-OMe,10,44,57,etb,33.616780,7.147286,101.224774
2,mof_unit_3,1089.818728,773.687960,788.50,0.14874,0.1262,OMe-COOH,2,22,24,pcu,19.263726,6.347967,118.987011
3,mof_unit_4,2205.198301,1304.638720,1441.53,0.21814,0.2220,H-SO3H,9,17,24,sra,25.701377,6.190085,187.626004
4,mof_unit_5,1137.800963,901.736120,0.00,0.07778,0.0591,NHMe-OH,2,1,22,pcu,30.001838,6.478063,79.210001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68608,mof_unit_68609,1188.302573,1001.700216,0.00,0.00000,0.0000,Pr-F,3,4,24,pcu,24.131770,NaN,-12.943652
68609,mof_unit_68610,1506.660363,1493.296496,0.00,0.01108,0.0000,SO3H,10,42,46,etb,6.071818,NaN,-12.985582
68610,mof_unit_68611,2035.532738,1959.518320,0.00,0.00000,0.0000,OPr,4,14,22,acs,9.876134,NaN,-13.187635
68611,mof_unit_68612,3985.426053,3638.677280,0.00,0.00000,0.0000,OPr-Me,4,4,15,acs,5.285051,inf,15.672698


## Extract data

In [123]:
base_dir = 'mof_cif_train'

In [125]:
train_cif_data = []
for i in tqdm(range(len(train_df))):
    a, b, c, alpha_r, beta_r, gamma_r, n_atoms, mol_avg_mass, charges, mol_avg_radius = get_cell_parameters(os.path.sep.join([base_dir,f'mof_unit_{i+1}.cif']))
    train_cif_data.append([a, b, c, alpha_r, beta_r, gamma_r, n_atoms, mol_avg_mass, charges, mol_avg_radius])

  1%|█                                                                             | 883/68613 [00:31<45:27, 24.83it/s]c:\users\admin\anaconda3\envs\chemistry-ml\lib\site-packages\ase\spacegroup\spacegroup.py:435: UserWarning: scaled_positions 95 and 96 are equivalent
  (kinds[ind], kind))
  2%|█▊                                                                           | 1629/68613 [01:01<36:15, 30.80it/s]c:\users\admin\anaconda3\envs\chemistry-ml\lib\site-packages\ase\spacegroup\spacegroup.py:435: UserWarning: scaled_positions 152 and 153 are equivalent
  (kinds[ind], kind))
  3%|█▉                                                                           | 1764/68613 [01:06<31:54, 34.92it/s]c:\users\admin\anaconda3\envs\chemistry-ml\lib\site-packages\ase\spacegroup\spacegroup.py:435: UserWarning: scaled_positions 174 and 175 are equivalent
  (kinds[ind], kind))
c:\users\admin\anaconda3\envs\chemistry-ml\lib\site-packages\ase\spacegroup\spacegroup.py:435: UserWarning: scaled_position

 14%|██████████▋                                                                  | 9495/68613 [06:03<43:40, 22.56it/s]c:\users\admin\anaconda3\envs\chemistry-ml\lib\site-packages\ase\spacegroup\spacegroup.py:435: UserWarning: scaled_positions 86 and 87 are equivalent
  (kinds[ind], kind))
 15%|███████████▏                                                                 | 9995/68613 [06:25<40:40, 24.02it/s]c:\users\admin\anaconda3\envs\chemistry-ml\lib\site-packages\ase\spacegroup\spacegroup.py:435: UserWarning: scaled_positions 177 and 178 are equivalent
  (kinds[ind], kind))
 17%|████████████▌                                                               | 11344/68613 [07:17<35:59, 26.52it/s]c:\users\admin\anaconda3\envs\chemistry-ml\lib\site-packages\ase\spacegroup\spacegroup.py:435: UserWarning: scaled_positions 175 and 176 are equivalent
  (kinds[ind], kind))
 17%|████████████▋                                                               | 11446/68613 [07:20<37:33, 25.37it/s]c:\u

 44%|█████████████████████████████████                                           | 29904/68613 [19:05<21:39, 29.78it/s]c:\users\admin\anaconda3\envs\chemistry-ml\lib\site-packages\ase\spacegroup\spacegroup.py:435: UserWarning: scaled_positions 190 and 191 are equivalent
  (kinds[ind], kind))
 44%|█████████████████████████████████▌                                          | 30343/68613 [19:22<28:16, 22.56it/s]c:\users\admin\anaconda3\envs\chemistry-ml\lib\site-packages\ase\spacegroup\spacegroup.py:435: UserWarning: scaled_positions 94 and 95 are equivalent
  (kinds[ind], kind))
 46%|███████████████████████████████████                                         | 31676/68613 [20:13<29:41, 20.73it/s]c:\users\admin\anaconda3\envs\chemistry-ml\lib\site-packages\ase\spacegroup\spacegroup.py:435: UserWarning: scaled_positions 241 and 242 are equivalent
  (kinds[ind], kind))
c:\users\admin\anaconda3\envs\chemistry-ml\lib\site-packages\ase\spacegroup\spacegroup.py:435: UserWarning: scaled_position

In [126]:
train_extra_feats = pd.DataFrame(train_cif_data, columns=['_cell_length_a', '_cell_length_b','_cell_length_c', 
                                                          '_cell_angle_alpha', '_cell_angle_beta','_cell_angle_gamma',
                                                          'n_atoms', 'mol_avg_mass', 'charges', 'mol_avg_radius'])

In [127]:
train_extra_feats

,_cell_length_a,_cell_length_b,_cell_length_c,_cell_angle_alpha,_cell_angle_beta,_cell_angle_gamma,n_atoms,mol_avg_mass,charges,mol_avg_radius
0,10.609882,10.643578,9.890832,1.569125,1.592480,1.575368,75,11.669907,92,59.653333
1,8.463295,17.684225,18.960098,1.746437,1.602488,1.691961,194,11.400559,250,60.515464
2,10.732110,9.552271,10.631996,1.556872,1.569806,1.577559,82,9.435293,120,59.585366
3,6.935530,17.504896,19.274980,1.911789,1.574891,1.580099,112,11.648598,204,63.500000
4,10.825925,9.699886,10.853274,1.565467,1.622999,1.595312,94,9.593000,90,58.425532
...,...,...,...,...,...,...,...,...,...,...
68608,10.718161,10.886490,10.193870,1.585497,1.609910,1.583947,119,8.417773,216,59.277311
68609,8.192620,12.576230,15.033794,1.661287,1.730445,1.700483,126,11.851548,126,62.460317
68610,11.237482,11.321902,18.608120,1.574297,1.572863,1.034849,204,9.605572,366,61.578431
68611,19.396341,11.081428,18.544746,1.578949,1.585477,1.569257,364,9.996454,652,62.054945


In [128]:
train_extra = pd.concat([train_df, train_extra_feats], axis=1)
train_extra

,MOFname,volume [A^3],weight [u],surface_area [m^2/g],void_fraction,void_volume [cm^3/g],functional_groups,metal_linker,organic_linker1,organic_linker2,...,_cell_length_a,_cell_length_b,_cell_length_c,_cell_angle_alpha,_cell_angle_beta,_cell_angle_gamma,n_atoms,mol_avg_mass,charges,mol_avg_radius
0,mof_unit_1,1116.667429,875.240600,0.00,0.07899,0.0607,COOH-OEt,3,4,11,...,10.609882,10.643578,9.890832,1.569125,1.592480,1.575368,75,11.669907,92,59.653333
1,mof_unit_2,2769.503842,2211.697211,603.61,0.13794,0.1040,F-OMe,10,44,57,...,8.463295,17.684225,18.960098,1.746437,1.602488,1.691961,194,11.400559,250,60.515464
2,mof_unit_3,1089.818728,773.687960,788.50,0.14874,0.1262,OMe-COOH,2,22,24,...,10.732110,9.552271,10.631996,1.556872,1.569806,1.577559,82,9.435293,120,59.585366
3,mof_unit_4,2205.198301,1304.638720,1441.53,0.21814,0.2220,H-SO3H,9,17,24,...,6.935530,17.504896,19.274980,1.911789,1.574891,1.580099,112,11.648598,204,63.500000
4,mof_unit_5,1137.800963,901.736120,0.00,0.07778,0.0591,NHMe-OH,2,1,22,...,10.825925,9.699886,10.853274,1.565467,1.622999,1.595312,94,9.593000,90,58.425532
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68608,mof_unit_68609,1188.302573,1001.700216,0.00,0.00000,0.0000,Pr-F,3,4,24,...,10.718161,10.886490,10.193870,1.585497,1.609910,1.583947,119,8.417773,216,59.277311
68609,mof_unit_68610,1506.660363,1493.296496,0.00,0.01108,0.0000,SO3H,10,42,46,...,8.192620,12.576230,15.033794,1.661287,1.730445,1.700483,126,11.851548,126,62.460317
68610,mof_unit_68611,2035.532738,1959.518320,0.00,0.00000,0.0000,OPr,4,14,22,...,11.237482,11.321902,18.608120,1.574297,1.572863,1.034849,204,9.605572,366,61.578431
68611,mof_unit_68612,3985.426053,3638.677280,0.00,0.00000,0.0000,OPr-Me,4,4,15,...,19.396341,11.081428,18.544746,1.578949,1.585477,1.569257,364,9.996454,652,62.054945


In [129]:
train_extra.to_csv('train_extra_2.csv', index=False)

# Test

In [130]:
test_df = pd.read_csv('test.csv')
test_df

,MOFname,volume [A^3],weight [u],surface_area [m^2/g],void_fraction,void_volume [cm^3/g],functional_groups,metal_linker,organic_linker1,organic_linker2,topology,CO2/N2_selectivity,heat_adsorption_CO2_P0.15bar_T298K [kcal/mol]
0,mof_unit_68614,1208.301332,797.70936,586.54,0.11392,0.1039,OEt,2,5,26,pcu,36.639791,7.005640
1,mof_unit_68615,4126.414623,3733.65779,852.49,0.21367,0.1422,H-I,4,6,17,acs,18.390691,5.119399
2,mof_unit_68616,1602.148373,747.21048,3155.73,0.33883,0.4375,CN-OH,3,11,17,pcu,13.062850,5.045400
3,mof_unit_68617,2436.629312,995.80232,3521.09,0.40464,0.5963,OMe,2,1,28,pcu,9.601198,5.106238
4,mof_unit_68618,3123.418006,1337.53800,2678.46,0.38959,0.5479,NO2-Pr,3,8,19,pcu,12.974954,5.287639
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16995,mof_unit_85609,32660.944605,4723.68288,5720.14,0.77614,3.2318,OH-NO2,3,3,14,nbo,4.536626,3.146698
16996,mof_unit_85610,5070.998617,1499.13262,4017.28,0.59192,1.2058,Me-OMe,3,1,11,nbo,6.745508,3.658871
16997,mof_unit_85611,4669.804446,1322.04892,4288.76,0.54950,1.1689,Me-CN,2,7,23,pcu,4.666206,3.593052
16998,mof_unit_85612,4682.120862,1213.51148,4331.86,0.60643,1.4091,OH-HCO,3,7,25,pcu,4.823305,3.454497


## Extract data

In [134]:
base_dir = 'mof_cif_test'

In [135]:
68613+len(test_df)

85613

In [136]:
test_cif_data = []
for i in tqdm(range(68613, 68613+len(test_df))):
    a, b, c, alpha_r, beta_r, gamma_r, n_atoms, mol_avg_mass, charges, mol_avg_radius = get_cell_parameters(os.path.sep.join([base_dir,f'mof_unit_{i+1}.cif']))
    test_cif_data.append([a, b, c, alpha_r, beta_r, gamma_r, n_atoms, mol_avg_mass, charges, mol_avg_radius])

 14%|██████████▋                                                                  | 2352/17000 [01:28<08:36, 28.38it/s]c:\users\admin\anaconda3\envs\chemistry-ml\lib\site-packages\ase\spacegroup\spacegroup.py:435: UserWarning: scaled_positions 103 and 104 are equivalent
  (kinds[ind], kind))
 63%|███████████████████████████████████████████████▌                            | 10638/17000 [06:39<12:33,  8.45it/s]c:\users\admin\anaconda3\envs\chemistry-ml\lib\site-packages\ase\spacegroup\spacegroup.py:435: UserWarning: scaled_positions 183 and 184 are equivalent
  (kinds[ind], kind))
 68%|███████████████████████████████████████████████████▊                        | 11579/17000 [07:13<03:47, 23.78it/s]c:\users\admin\anaconda3\envs\chemistry-ml\lib\site-packages\ase\io\cif.py:404: UserWarning: crystal system 'tetragonal' is not interpreted for space group 1. This may result in wrong setting!
  setting_name, spacegroup))
 83%|███████████████████████████████████████████████████████████████     

In [137]:
test_extra_feats = pd.DataFrame(test_cif_data, columns=['_cell_length_a', '_cell_length_b','_cell_length_c', 
                                                          '_cell_angle_alpha', '_cell_angle_beta','_cell_angle_gamma',
                                                          'n_atoms', 'mol_avg_mass', 'charges', 'mol_avg_radius'])

In [138]:
test_extra_feats

,_cell_length_a,_cell_length_b,_cell_length_c,_cell_angle_alpha,_cell_angle_beta,_cell_angle_gamma,n_atoms,mol_avg_mass,charges,mol_avg_radius
0,7.806778,9.084327,17.070225,1.556313,1.511002,1.575256,84,9.496619,128,59.761905
1,12.951336,13.273001,27.655520,1.656643,1.574169,2.081566,158,23.630859,314,70.329114
2,14.948982,10.697065,10.019137,1.569684,1.574747,1.571707,60,12.453633,94,62.433333
3,10.835044,9.587345,23.461267,1.565566,1.590575,1.570093,94,10.593745,160,60.787234
4,11.498125,16.343084,16.653784,1.517304,1.599094,1.587196,118,11.335136,94,58.711864
...,...,...,...,...,...,...,...,...,...,...
16995,44.634366,38.924145,21.601534,1.565134,1.055786,1.568475,416,11.355106,768,61.711538
16996,18.653876,18.775583,18.721517,1.905919,1.908772,1.910598,124,12.089839,196,62.201613
16997,17.736994,16.514905,15.943922,1.570403,1.586349,1.570638,112,11.804214,198,62.821429
16998,15.773696,17.587298,16.887609,1.571806,1.571627,1.605246,104,11.668462,184,60.884615


In [139]:
test_extra = pd.concat([test_df, test_extra_feats], axis=1)
test_extra

,MOFname,volume [A^3],weight [u],surface_area [m^2/g],void_fraction,void_volume [cm^3/g],functional_groups,metal_linker,organic_linker1,organic_linker2,...,_cell_length_a,_cell_length_b,_cell_length_c,_cell_angle_alpha,_cell_angle_beta,_cell_angle_gamma,n_atoms,mol_avg_mass,charges,mol_avg_radius
0,mof_unit_68614,1208.301332,797.70936,586.54,0.11392,0.1039,OEt,2,5,26,...,7.806778,9.084327,17.070225,1.556313,1.511002,1.575256,84,9.496619,128,59.761905
1,mof_unit_68615,4126.414623,3733.65779,852.49,0.21367,0.1422,H-I,4,6,17,...,12.951336,13.273001,27.655520,1.656643,1.574169,2.081566,158,23.630859,314,70.329114
2,mof_unit_68616,1602.148373,747.21048,3155.73,0.33883,0.4375,CN-OH,3,11,17,...,14.948982,10.697065,10.019137,1.569684,1.574747,1.571707,60,12.453633,94,62.433333
3,mof_unit_68617,2436.629312,995.80232,3521.09,0.40464,0.5963,OMe,2,1,28,...,10.835044,9.587345,23.461267,1.565566,1.590575,1.570093,94,10.593745,160,60.787234
4,mof_unit_68618,3123.418006,1337.53800,2678.46,0.38959,0.5479,NO2-Pr,3,8,19,...,11.498125,16.343084,16.653784,1.517304,1.599094,1.587196,118,11.335136,94,58.711864
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16995,mof_unit_85609,32660.944605,4723.68288,5720.14,0.77614,3.2318,OH-NO2,3,3,14,...,44.634366,38.924145,21.601534,1.565134,1.055786,1.568475,416,11.355106,768,61.711538
16996,mof_unit_85610,5070.998617,1499.13262,4017.28,0.59192,1.2058,Me-OMe,3,1,11,...,18.653876,18.775583,18.721517,1.905919,1.908772,1.910598,124,12.089839,196,62.201613
16997,mof_unit_85611,4669.804446,1322.04892,4288.76,0.54950,1.1689,Me-CN,2,7,23,...,17.736994,16.514905,15.943922,1.570403,1.586349,1.570638,112,11.804214,198,62.821429
16998,mof_unit_85612,4682.120862,1213.51148,4331.86,0.60643,1.4091,OH-HCO,3,7,25,...,15.773696,17.587298,16.887609,1.571806,1.571627,1.605246,104,11.668462,184,60.884615


In [140]:
test_extra.to_csv('test_extra_2.csv', index=False)

---

# Atoms volume

In [2]:
df = pd.read_csv('atom_mass_vol.csv')
df

,Element,Energy level,Mass(u),Change,Radius(pm),Radius(A),Volume(A^3)
0,H,1,1.0080,1,53,0.53,0.623865524
1,He,1,4.0026,0,31,0.31,0.124838476
2,Li,2,6.9400,1,167,1.67,19.51698781
3,Be,2,9.0122,2,112,1.12,5.887317333
4,B,2,10.8100,3,87,0.87,2.759441143
...,...,...,...,...,...,...,...
113,Fi,7,289.0000,0,-,-,-
114,Mc,7,290.0000,0,-,-,-
115,Lv,7,293.0000,0,-,-,-
116,Ts,7,294.0000,0,-,-,-


In [3]:
df = df.set_index('Element', drop=True)
vol_df = df['Volume(A^3)'].replace({'-':0}).astype(float)

In [4]:
vol_dict = vol_df.to_dict()
vol_dict

{'H': 0.623865524,
 'He': 0.124838476,
 'Li': 19.51698781,
 'Be': 5.887317333,
 'B': 2.759441143,
 'C': 1.26034019,
 'N': 0.735914667,
 'O': 0.463433143,
 'F': 0.310464,
 'Ne': 0.22993981,
 'Na': 28.74247619,
 'Mg': 12.77519048,
 'Al': 6.885086476,
 'Si': 5.731025143,
 'P': 3.944042667,
 'S': 2.85569219,
 'Cl': 2.06606819,
 'Ar': 1.499817524,
 'K': 60.12875314,
 'Ca': 30.59627581,
 'Sc': 26.10458819,
 'Ti': 15.90879086,
 'V': 20.95326514,
 'Cr': 19.16847848,
 'Mn': 17.48803467,
 'Fe': 15.90879086,
 'Co': 14.71614781,
 'Ni': 13.86188152,
 'Cu': 12.77519048,
 'Zn': 11.99854019,
 'Ga': 10.54095848,
 'Ge': 8.18452381,
 'As': 6.208374857,
 'Se': 4.579046476,
 'Br': 3.480542476,
 'Kr': 2.85569219,
 'Rb': 77.98319048,
 'Sr': 44.01449486,
 'Y': 39.92739352,
 'Zr': 36.63237181,
 'Nb': 32.52811886,
 'Mo': 28.74247619,
 'Tc': 25.68127886,
 'Ru': 23.63324648,
 'Rh': 21.69709981,
 'Pd': 20.22662819,
 'Ag': 18.82414286,
 'Cd': 17.48803467,
 'In': 15.90879086,
 'Sn': 12.77519048,
 'Sb': 9.858669333,


In [11]:
calc_atoms_vol('mof_cif_train/mof_unit_1.cif', vol_dict)

Counter({'C': 30, 'O': 21, 'H': 20, 'Zn': 2, 'N': 2})


85.488521897

---

# Area (A^2)

In [3]:
df = pd.read_csv('atom_mass_vol_area.csv')
df

,Element,Energy level,Mass(u),Change,Radius(pm),Radius(A),Volume(A^3),Area(A^2)
0,H,1,1.0080,1,53,0.53,0.623865524,3.531314286
1,He,1,4.0026,0,31,0.31,0.124838476,1.208114286
2,Li,2,6.9400,1,167,1.67,19.51698781,35.06045714
3,Be,2,9.0122,2,112,1.12,5.887317333,15.7696
4,B,2,10.8100,3,87,0.87,2.759441143,9.515314286
...,...,...,...,...,...,...,...,...
113,Fi,7,289.0000,0,-,-,-,#VALUE!
114,Mc,7,290.0000,0,-,-,-,#VALUE!
115,Lv,7,293.0000,0,-,-,-,#VALUE!
116,Ts,7,294.0000,0,-,-,-,#VALUE!


In [4]:
df = df.set_index('Element', drop=True)
area_df = df['Area(A^2)'].replace({'#VALUE!':0}).astype(float)

In [5]:
area_dict = area_df.to_dict()
area_dict

{'H': 3.531314286,
 'He': 1.208114286,
 'Li': 35.06045714,
 'Be': 15.7696,
 'B': 9.515314286,
 'C': 5.643314286,
 'N': 3.9424,
 'O': 2.896457143,
 'F': 2.2176,
 'Ne': 1.815314286,
 'Na': 45.38285714,
 'Mg': 26.43142857,
 'Al': 17.50445714,
 'Si': 15.48925714,
 'P': 12.0736,
 'S': 9.735314286,
 'Cl': 7.845828571,
 'Ar': 6.337257143,
 'K': 74.23302857,
 'Ca': 47.31382857,
 'Sc': 42.56182857,
 'Ti': 30.59382857,
 'V': 36.76011429,
 'Cr': 34.64182857,
 'Mn': 32.5864,
 'Fe': 30.59382857,
 'Co': 29.04502857,
 'Ni': 27.90982857,
 'Cu': 26.43142857,
 'Zn': 25.34902857,
 'Ga': 23.25211429,
 'Ge': 19.64285714,
 'As': 16.33782857,
 'Se': 13.33702857,
 'Br': 11.10811429,
 'Kr': 9.735314286,
 'Rb': 88.28285714,
 'Sr': 60.29382857,
 'Y': 56.50102857,
 'Zr': 53.34811429,
 'Nb': 49.28502857,
 'Mo': 45.38285714,
 'Tc': 42.10045714,
 'Ru': 39.83131429,
 'Rh': 37.62502857,
 'Pd': 35.90525714,
 'Ag': 34.22571429,
 'Cd': 32.5864,
 'In': 30.59382857,
 'Sn': 26.43142857,
 'Sb': 22.2376,
 'Te': 19.01931429,
 

In [7]:
calc_atoms_area('mof_cif_train/mof_unit_1.cif', area_dict)

c:\users\admin\anaconda3\envs\chemistry-ml\lib\site-packages\ase\io\cif.py:404: UserWarning: crystal system 'triclinic' is not interpreted for space group 1. This may result in wrong setting!
  setting_name, spacegroup))


359.33417144299995

## Train

In [8]:
train_df = pd.read_csv('train_extra_2.csv')
train_df

,MOFname,volume [A^3],weight [u],surface_area [m^2/g],void_fraction,void_volume [cm^3/g],functional_groups,metal_linker,organic_linker1,organic_linker2,...,_cell_length_b,_cell_length_c,_cell_angle_alpha,_cell_angle_beta,_cell_angle_gamma,n_atoms,mol_avg_mass,charges,mol_avg_radius,atoms_volume
0,mof_unit_1,1116.667429,875.240600,0.00,0.07899,0.0607,COOH-OEt,3,4,11,...,10.643578,9.890832,1.569125,1.592480,1.575368,75,11.669907,92,59.653333,85.488522
1,mof_unit_2,2769.503842,2211.697211,603.61,0.13794,0.1040,F-OMe,10,44,57,...,17.684225,18.960098,1.746437,1.602488,1.691961,194,11.400559,250,60.515464,318.194213
2,mof_unit_3,1089.818728,773.687960,788.50,0.14874,0.1262,OMe-COOH,2,22,24,...,9.552271,10.631996,1.556872,1.569806,1.577559,82,9.435293,120,59.585366,92.531908
3,mof_unit_4,2205.198301,1304.638720,1441.53,0.21814,0.2220,H-SO3H,9,17,24,...,17.504896,19.274980,1.911789,1.574891,1.580099,112,11.648598,204,63.500000,185.375000
4,mof_unit_5,1137.800963,901.736120,0.00,0.07778,0.0591,NHMe-OH,2,1,22,...,9.699886,10.853274,1.565467,1.622999,1.595312,94,9.593000,90,58.425532,98.775910
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68608,mof_unit_68609,1188.302573,1001.700216,0.00,0.00000,0.0000,Pr-F,3,4,24,...,10.886490,10.193870,1.585497,1.609910,1.583947,119,8.417773,216,59.277311,124.277288
68609,mof_unit_68610,1506.660363,1493.296496,0.00,0.01108,0.0000,SO3H,10,42,46,...,12.576230,15.033794,1.661287,1.730445,1.700483,126,11.851548,126,62.460317,261.730396
68610,mof_unit_68611,2035.532738,1959.518320,0.00,0.00000,0.0000,OPr,4,14,22,...,11.321902,18.608120,1.574297,1.572863,1.034849,204,9.605572,366,61.578431,289.412179
68611,mof_unit_68612,3985.426053,3638.677280,0.00,0.00000,0.0000,OPr-Me,4,4,15,...,11.081428,18.544746,1.578949,1.585477,1.569257,364,9.996454,652,62.054945,541.832410


In [9]:
base_dir = 'mof_cif_train'

In [10]:
train_cif_data = []
for i in tqdm(range(len(train_df))):
    atoms_area = calc_atoms_area(os.path.sep.join([base_dir,f'mof_unit_{i+1}.cif']), area_dict)
    train_cif_data.append(atoms_area)

  0%|                                                                                        | 0/68613 [00:00<?, ?it/s]c:\users\admin\anaconda3\envs\chemistry-ml\lib\site-packages\ase\io\cif.py:404: UserWarning: crystal system 'triclinic' is not interpreted for space group 1. This may result in wrong setting!
  setting_name, spacegroup))
  0%|                                                                               | 65/68613 [00:02<48:33, 23.53it/s]c:\users\admin\anaconda3\envs\chemistry-ml\lib\site-packages\ase\io\cif.py:404: UserWarning: crystal system 'monoclinic' is not interpreted for space group 1. This may result in wrong setting!
  setting_name, spacegroup))
  0%|                                                                               | 78/68613 [00:03<47:00, 24.30it/s]c:\users\admin\anaconda3\envs\chemistry-ml\lib\site-packages\ase\spacegroup\spacegroup.py:435: UserWarning: scaled_positions 165 and 166 are equivalent
  (kinds[ind], kind))
  1%|█                    

 13%|█████████▋                                                                   | 8669/68613 [06:12<46:18, 21.58it/s]c:\users\admin\anaconda3\envs\chemistry-ml\lib\site-packages\ase\spacegroup\spacegroup.py:435: UserWarning: scaled_positions 123 and 124 are equivalent
  (kinds[ind], kind))
c:\users\admin\anaconda3\envs\chemistry-ml\lib\site-packages\ase\spacegroup\spacegroup.py:435: UserWarning: scaled_positions 132 and 133 are equivalent
  (kinds[ind], kind))
c:\users\admin\anaconda3\envs\chemistry-ml\lib\site-packages\ase\spacegroup\spacegroup.py:435: UserWarning: scaled_positions 135 and 136 are equivalent
  (kinds[ind], kind))
 13%|██████████▏                                                                  | 9075/68613 [06:28<40:36, 24.44it/s]c:\users\admin\anaconda3\envs\chemistry-ml\lib\site-packages\ase\spacegroup\spacegroup.py:435: UserWarning: scaled_positions 155 and 156 are equivalent
  (kinds[ind], kind))
 14%|██████████▋                                                  

 43%|████████████████████████████████▌                                           | 29439/68613 [20:58<27:01, 24.15it/s]c:\users\admin\anaconda3\envs\chemistry-ml\lib\site-packages\ase\spacegroup\spacegroup.py:435: UserWarning: scaled_positions 78 and 79 are equivalent
  (kinds[ind], kind))
 43%|█████████████████████████████████                                           | 29838/68613 [21:12<20:24, 31.66it/s]c:\users\admin\anaconda3\envs\chemistry-ml\lib\site-packages\ase\spacegroup\spacegroup.py:435: UserWarning: scaled_positions 249 and 250 are equivalent
  (kinds[ind], kind))
c:\users\admin\anaconda3\envs\chemistry-ml\lib\site-packages\ase\spacegroup\spacegroup.py:435: UserWarning: scaled_positions 252 and 253 are equivalent
  (kinds[ind], kind))
c:\users\admin\anaconda3\envs\chemistry-ml\lib\site-packages\ase\spacegroup\spacegroup.py:435: UserWarning: scaled_positions 255 and 256 are equivalent
  (kinds[ind], kind))
c:\users\admin\anaconda3\envs\chemistry-ml\lib\site-packages\ase\spa

 70%|████████████████████████████████████████████████████▉                       | 47752/68613 [33:05<19:52, 17.49it/s]c:\users\admin\anaconda3\envs\chemistry-ml\lib\site-packages\ase\spacegroup\spacegroup.py:435: UserWarning: scaled_positions 158 and 159 are equivalent
  (kinds[ind], kind))
 70%|█████████████████████████████████████████████████████▍                      | 48200/68613 [33:24<11:07, 30.58it/s]c:\users\admin\anaconda3\envs\chemistry-ml\lib\site-packages\ase\spacegroup\spacegroup.py:435: UserWarning: scaled_positions 71 and 72 are equivalent
  (kinds[ind], kind))
 73%|███████████████████████████████████████████████████████▏                    | 49806/68613 [34:30<12:33, 24.95it/s]c:\users\admin\anaconda3\envs\chemistry-ml\lib\site-packages\ase\spacegroup\spacegroup.py:435: UserWarning: scaled_positions 91 and 92 are equivalent
  (kinds[ind], kind))
 74%|███████████████████████████████████████████████████████▉                    | 50537/68613 [35:00<15:14, 19.77it/s]c:\use

In [11]:
train_extra_feats = pd.DataFrame(train_cif_data, columns=['atoms_area'])

In [12]:
train_extra_feats

,atoms_area
0,359.334171
1,1020.568686
2,390.662171
3,638.296686
4,428.741029
...,...
68608,552.300571
68609,736.477029
68610,1072.591771
68611,1958.553143


In [13]:
train_extra = pd.concat([train_df, train_extra_feats], axis=1)
train_extra

,MOFname,volume [A^3],weight [u],surface_area [m^2/g],void_fraction,void_volume [cm^3/g],functional_groups,metal_linker,organic_linker1,organic_linker2,...,_cell_length_c,_cell_angle_alpha,_cell_angle_beta,_cell_angle_gamma,n_atoms,mol_avg_mass,charges,mol_avg_radius,atoms_volume,atoms_area
0,mof_unit_1,1116.667429,875.240600,0.00,0.07899,0.0607,COOH-OEt,3,4,11,...,9.890832,1.569125,1.592480,1.575368,75,11.669907,92,59.653333,85.488522,359.334171
1,mof_unit_2,2769.503842,2211.697211,603.61,0.13794,0.1040,F-OMe,10,44,57,...,18.960098,1.746437,1.602488,1.691961,194,11.400559,250,60.515464,318.194213,1020.568686
2,mof_unit_3,1089.818728,773.687960,788.50,0.14874,0.1262,OMe-COOH,2,22,24,...,10.631996,1.556872,1.569806,1.577559,82,9.435293,120,59.585366,92.531908,390.662171
3,mof_unit_4,2205.198301,1304.638720,1441.53,0.21814,0.2220,H-SO3H,9,17,24,...,19.274980,1.911789,1.574891,1.580099,112,11.648598,204,63.500000,185.375000,638.296686
4,mof_unit_5,1137.800963,901.736120,0.00,0.07778,0.0591,NHMe-OH,2,1,22,...,10.853274,1.565467,1.622999,1.595312,94,9.593000,90,58.425532,98.775910,428.741029
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68608,mof_unit_68609,1188.302573,1001.700216,0.00,0.00000,0.0000,Pr-F,3,4,24,...,10.193870,1.585497,1.609910,1.583947,119,8.417773,216,59.277311,124.277288,552.300571
68609,mof_unit_68610,1506.660363,1493.296496,0.00,0.01108,0.0000,SO3H,10,42,46,...,15.033794,1.661287,1.730445,1.700483,126,11.851548,126,62.460317,261.730396,736.477029
68610,mof_unit_68611,2035.532738,1959.518320,0.00,0.00000,0.0000,OPr,4,14,22,...,18.608120,1.574297,1.572863,1.034849,204,9.605572,366,61.578431,289.412179,1072.591771
68611,mof_unit_68612,3985.426053,3638.677280,0.00,0.00000,0.0000,OPr-Me,4,4,15,...,18.544746,1.578949,1.585477,1.569257,364,9.996454,652,62.054945,541.832410,1958.553143


In [14]:
train_extra.to_csv('train_extra_2.csv', index=False)

## Test

In [15]:
test_df = pd.read_csv('test_extra_2.csv')
test_df

,MOFname,volume [A^3],weight [u],surface_area [m^2/g],void_fraction,void_volume [cm^3/g],functional_groups,metal_linker,organic_linker1,organic_linker2,...,_cell_length_b,_cell_length_c,_cell_angle_alpha,_cell_angle_beta,_cell_angle_gamma,n_atoms,mol_avg_mass,charges,mol_avg_radius,atoms_volume
0,mof_unit_68614,1208.301332,797.70936,586.54,0.11392,0.1039,OEt,2,5,26,...,9.084327,17.070225,1.556313,1.511002,1.575256,84,9.496619,128,59.761905,95.052588
1,mof_unit_68615,4126.414623,3733.65779,852.49,0.21367,0.1422,H-I,4,6,17,...,13.273001,27.655520,1.656643,1.574169,2.081566,158,23.630859,314,70.329114,349.189783
2,mof_unit_68616,1602.148373,747.21048,3155.73,0.33883,0.4375,CN-OH,3,11,17,...,10.697065,10.019137,1.569684,1.574747,1.571707,60,12.453633,94,62.433333,78.343492
3,mof_unit_68617,2436.629312,995.80232,3521.09,0.40464,0.5963,OMe,2,1,28,...,9.587345,23.461267,1.565566,1.590575,1.570093,94,10.593745,160,60.787234,108.832173
4,mof_unit_68618,3123.418006,1337.53800,2678.46,0.38959,0.5479,NO2-Pr,3,8,19,...,16.343084,16.653784,1.517304,1.599094,1.587196,118,11.335136,94,58.711864,120.398097
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16995,mof_unit_85609,32660.944605,4723.68288,5720.14,0.77614,3.2318,OH-NO2,3,3,14,...,38.924145,21.601534,1.565134,1.055786,1.568475,416,11.355106,768,61.711538,517.114417
16996,mof_unit_85610,5070.998617,1499.13262,4017.28,0.59192,1.2058,Me-OMe,3,1,11,...,18.775583,18.721517,1.905919,1.908772,1.910598,124,12.089839,196,62.201613,175.008856
16997,mof_unit_85611,4669.804446,1322.04892,4288.76,0.54950,1.1689,Me-CN,2,7,23,...,16.514905,15.943922,1.570403,1.586349,1.570638,112,11.804214,198,62.821429,135.867491
16998,mof_unit_85612,4682.120862,1213.51148,4331.86,0.60643,1.4091,OH-HCO,3,7,25,...,17.587298,16.887609,1.571806,1.571627,1.605246,104,11.668462,184,60.884615,118.374717


In [16]:
base_dir = 'mof_cif_test'

In [17]:
test_cif_data = []
for i in tqdm(range(68613, 68613+len(test_df))):
    atoms_area = calc_atoms_area(os.path.sep.join([base_dir,f'mof_unit_{i+1}.cif']), area_dict)
    test_cif_data.append(atoms_area)

  0%|                                                                                        | 0/17000 [00:00<?, ?it/s]c:\users\admin\anaconda3\envs\chemistry-ml\lib\site-packages\ase\io\cif.py:404: UserWarning: crystal system 'triclinic' is not interpreted for space group 1. This may result in wrong setting!
  setting_name, spacegroup))
  0%|▏                                                                              | 44/17000 [00:01<11:10, 25.30it/s]c:\users\admin\anaconda3\envs\chemistry-ml\lib\site-packages\ase\io\cif.py:404: UserWarning: crystal system 'monoclinic' is not interpreted for space group 1. This may result in wrong setting!
  setting_name, spacegroup))
  6%|████▋                                                                        | 1042/17000 [00:41<08:19, 31.96it/s]c:\users\admin\anaconda3\envs\chemistry-ml\lib\site-packages\ase\spacegroup\spacegroup.py:435: UserWarning: scaled_positions 180 and 181 are equivalent
  (kinds[ind], kind))
c:\users\admin\anaconda3\e

In [18]:
test_extra_feats = pd.DataFrame(test_cif_data, columns=['atoms_area'])

In [19]:
test_extra_feats

,atoms_area
0,401.948800
1,1114.600457
2,314.992229
3,462.158400
4,537.835886
...,...
16995,2125.174857
16996,663.584114
16997,579.618286
16998,510.525714


In [20]:
test_extra = pd.concat([test_df, test_extra_feats], axis=1)
test_extra

,MOFname,volume [A^3],weight [u],surface_area [m^2/g],void_fraction,void_volume [cm^3/g],functional_groups,metal_linker,organic_linker1,organic_linker2,...,_cell_length_c,_cell_angle_alpha,_cell_angle_beta,_cell_angle_gamma,n_atoms,mol_avg_mass,charges,mol_avg_radius,atoms_volume,atoms_area
0,mof_unit_68614,1208.301332,797.70936,586.54,0.11392,0.1039,OEt,2,5,26,...,17.070225,1.556313,1.511002,1.575256,84,9.496619,128,59.761905,95.052588,401.948800
1,mof_unit_68615,4126.414623,3733.65779,852.49,0.21367,0.1422,H-I,4,6,17,...,27.655520,1.656643,1.574169,2.081566,158,23.630859,314,70.329114,349.189783,1114.600457
2,mof_unit_68616,1602.148373,747.21048,3155.73,0.33883,0.4375,CN-OH,3,11,17,...,10.019137,1.569684,1.574747,1.571707,60,12.453633,94,62.433333,78.343492,314.992229
3,mof_unit_68617,2436.629312,995.80232,3521.09,0.40464,0.5963,OMe,2,1,28,...,23.461267,1.565566,1.590575,1.570093,94,10.593745,160,60.787234,108.832173,462.158400
4,mof_unit_68618,3123.418006,1337.53800,2678.46,0.38959,0.5479,NO2-Pr,3,8,19,...,16.653784,1.517304,1.599094,1.587196,118,11.335136,94,58.711864,120.398097,537.835886
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16995,mof_unit_85609,32660.944605,4723.68288,5720.14,0.77614,3.2318,OH-NO2,3,3,14,...,21.601534,1.565134,1.055786,1.568475,416,11.355106,768,61.711538,517.114417,2125.174857
16996,mof_unit_85610,5070.998617,1499.13262,4017.28,0.59192,1.2058,Me-OMe,3,1,11,...,18.721517,1.905919,1.908772,1.910598,124,12.089839,196,62.201613,175.008856,663.584114
16997,mof_unit_85611,4669.804446,1322.04892,4288.76,0.54950,1.1689,Me-CN,2,7,23,...,15.943922,1.570403,1.586349,1.570638,112,11.804214,198,62.821429,135.867491,579.618286
16998,mof_unit_85612,4682.120862,1213.51148,4331.86,0.60643,1.4091,OH-HCO,3,7,25,...,16.887609,1.571806,1.571627,1.605246,104,11.668462,184,60.884615,118.374717,510.525714


In [21]:
test_extra.to_csv('test_extra_2.csv', index=False)

---

# Get Description

## Metal linker

In [54]:
base_dir = 'metal_linker_xyz'

In [55]:
metal_len = len(os.listdir(base_dir))
metal_len

12

In [60]:
metal_linker_data = []
for i in tqdm(range(metal_len)):
    hba1, hba2, hbd, mw, mlinker_atoms = get_desc_xyz(os.path.sep.join([base_dir,f'metal_linker_{i+1}.xyz']))
    metal_linker_data.append([hba1, hba2, hbd, mw, mlinker_atoms])

100%|█████████████████████████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 118.82it/s]


In [61]:
metal_linker_data

[[17.0, 13.0, 0.0, 541.5763999999998, 23],
 [10.0, 8.0, 0.0, 303.12999999999994, 14],
 [10.0, 8.0, 0.0, 306.79799999999994, 14],
 [16.0, 13.0, 0.0, 436.04470000000003, 22],
 [20.0, 16.0, 0.0, 801.72, 28],
 [12.0, 24.0, 0.0, 764.0521960000004, 44],
 [42.0, 32.0, 4.0, 1207.4849600000005, 54],
 [15.0, 10.0, 2.0, 290.9972939999999, 19],
 [12.0, 10.0, 0.0, 309.9198, 16],
 [12.0, 10.0, 0.0, 434.64799999999997, 12],
 [9.0, 8.0, 0.0, 290.856, 13],
 [36.0, 16.0, 0.0, 763.2646400000002, 60]]

In [62]:
metal_df = pd.DataFrame(data=metal_linker_data, columns=['HBA1', 'HBA2', 'HBD', 'MW', 'mlinker_atoms'], index=range(1,13))
metal_df

,HBA1,HBA2,HBD,MW,mlinker_atoms
1,17.0,13.0,0.0,541.576400,23
2,10.0,8.0,0.0,303.130000,14
3,10.0,8.0,0.0,306.798000,14
4,16.0,13.0,0.0,436.044700,22
5,20.0,16.0,0.0,801.720000,28
6,12.0,24.0,0.0,764.052196,44
7,42.0,32.0,4.0,1207.484960,54
8,15.0,10.0,2.0,290.997294,19
9,12.0,10.0,0.0,309.919800,16
10,12.0,10.0,0.0,434.648000,12


In [63]:
metal_df.to_csv('metal_linker_data.csv', index=True)

## Organic linker

In [64]:
base_dir = 'organic_linker_xyz'

In [65]:
organic_len = len(os.listdir(base_dir))
organic_len

59

In [66]:
organic_linker_data = []
for i in tqdm(range(organic_len)):
    hba1, hba2, hbd, mw, olinker_atoms = get_desc_xyz(os.path.sep.join([base_dir,f'organic_linker_{i+1}.xyz']))
    organic_linker_data.append([hba1, hba2, hbd, mw, olinker_atoms])

100%|██████████████████████████████████████████████████████████████████████████████████| 59/59 [00:00<00:00, 80.93it/s]


In [67]:
metal_linker_data

[[17.0, 13.0, 0.0, 541.5763999999998, 23],
 [10.0, 8.0, 0.0, 303.12999999999994, 14],
 [10.0, 8.0, 0.0, 306.79799999999994, 14],
 [16.0, 13.0, 0.0, 436.04470000000003, 22],
 [20.0, 16.0, 0.0, 801.72, 28],
 [12.0, 24.0, 0.0, 764.0521960000004, 44],
 [42.0, 32.0, 4.0, 1207.4849600000005, 54],
 [15.0, 10.0, 2.0, 290.9972939999999, 19],
 [12.0, 10.0, 0.0, 309.9198, 16],
 [12.0, 10.0, 0.0, 434.64799999999997, 12],
 [9.0, 8.0, 0.0, 290.856, 13],
 [36.0, 16.0, 0.0, 763.2646400000002, 60]]

In [69]:
organic_df = pd.DataFrame(data=organic_linker_data, columns=['HBA1', 'HBA2', 'HBD', 'MW', 'olinker_atoms'], index=range(1,organic_len+1))
organic_df

,HBA1,HBA2,HBD,MW,olinker_atoms
1,4.0,0.0,0.0,76.09596,10
2,8.0,0.0,0.0,152.19192,20
3,12.0,0.0,0.0,228.28788,30
4,6.0,0.0,0.0,102.13324,14
5,0.0,0.0,0.0,24.02140,2
6,4.0,0.0,0.0,124.13876,14
7,8.0,0.0,0.0,176.21332,22
8,10.0,2.0,0.0,180.20532,22
9,12.0,0.0,0.0,228.28788,30
10,2.0,0.0,0.0,26.03728,4


In [70]:
organic_df.to_csv('organic_linker_data.csv', index=True)

## Train

In [72]:
train_df = pd.read_csv('train_extra_2.csv')
train_df

,MOFname,volume [A^3],weight [u],surface_area [m^2/g],void_fraction,void_volume [cm^3/g],functional_groups,metal_linker,organic_linker1,organic_linker2,...,_cell_length_b,_cell_length_c,_cell_angle_alpha,_cell_angle_beta,_cell_angle_gamma,n_atoms,mol_avg_mass,charges,mol_avg_radius,atoms_volume
0,mof_unit_1,1116.667429,875.240600,0.00,0.07899,0.0607,COOH-OEt,3,4,11,...,10.643578,9.890832,1.569125,1.592480,1.575368,75,11.669907,92,59.653333,85.488522
1,mof_unit_2,2769.503842,2211.697211,603.61,0.13794,0.1040,F-OMe,10,44,57,...,17.684225,18.960098,1.746437,1.602488,1.691961,194,11.400559,250,60.515464,318.194213
2,mof_unit_3,1089.818728,773.687960,788.50,0.14874,0.1262,OMe-COOH,2,22,24,...,9.552271,10.631996,1.556872,1.569806,1.577559,82,9.435293,120,59.585366,92.531908
3,mof_unit_4,2205.198301,1304.638720,1441.53,0.21814,0.2220,H-SO3H,9,17,24,...,17.504896,19.274980,1.911789,1.574891,1.580099,112,11.648598,204,63.500000,185.375000
4,mof_unit_5,1137.800963,901.736120,0.00,0.07778,0.0591,NHMe-OH,2,1,22,...,9.699886,10.853274,1.565467,1.622999,1.595312,94,9.593000,90,58.425532,98.775910
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68608,mof_unit_68609,1188.302573,1001.700216,0.00,0.00000,0.0000,Pr-F,3,4,24,...,10.886490,10.193870,1.585497,1.609910,1.583947,119,8.417773,216,59.277311,124.277288
68609,mof_unit_68610,1506.660363,1493.296496,0.00,0.01108,0.0000,SO3H,10,42,46,...,12.576230,15.033794,1.661287,1.730445,1.700483,126,11.851548,126,62.460317,261.730396
68610,mof_unit_68611,2035.532738,1959.518320,0.00,0.00000,0.0000,OPr,4,14,22,...,11.321902,18.608120,1.574297,1.572863,1.034849,204,9.605572,366,61.578431,289.412179
68611,mof_unit_68612,3985.426053,3638.677280,0.00,0.00000,0.0000,OPr-Me,4,4,15,...,11.081428,18.544746,1.578949,1.585477,1.569257,364,9.996454,652,62.054945,541.832410


In [73]:
base_dir = 'mof_cif_train'

In [74]:
train_cif_desc = []
for i in tqdm(range(len(train_df))):
    hba1, hba2, hbd, nF, logP, MR, TPSA = get_desc_cif(os.path.sep.join([base_dir,f'mof_unit_{i+1}.cif']))
    train_cif_desc.append([hba1, hba2, hbd, nF, logP, MR, TPSA])

100%|████████████████████████████████████████████████████████████████████████████| 68613/68613 [37:50<00:00, 30.21it/s]


In [75]:
train_extra_desc = pd.DataFrame(train_cif_desc, columns=['hba1', 'hba2', 'hbd', 'nF', 'logP', 'MR', 'TPSA'])

In [76]:
train_extra_desc

,hba1,hba2,hbd,nF,logP,MR,TPSA
0,45.0,23.0,6.0,0.0,0.0,0.0,0.0
1,106.0,35.0,0.0,11.0,0.0,0.0,0.0
2,52.0,14.0,2.0,0.0,0.0,0.0,0.0
3,55.0,23.0,1.0,0.0,0.0,0.0,0.0
4,66.0,26.0,16.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
68608,67.0,10.0,0.0,5.0,0.0,0.0,0.0
68609,83.0,28.0,1.0,0.0,0.0,0.0,0.0
68610,116.0,32.0,0.0,0.0,0.0,0.0,0.0
68611,204.0,60.0,0.0,0.0,0.0,0.0,0.0


In [77]:
train_extra_desc.describe()

,hba1,hba2,hbd,nF,logP,MR,TPSA
count,68613.000000,68613.000000,68613.000000,68613.000000,68613.0,68613.0,68613.0
mean,74.307755,26.261292,3.024689,0.824173,0.0,0.0,0.0
std,56.098649,21.102319,6.228315,3.846238,0.0,0.0,0.0
min,10.000000,4.000000,0.000000,0.000000,0.0,0.0,0.0
25%,40.000000,13.000000,0.000000,0.000000,0.0,0.0,0.0
50%,57.000000,20.000000,0.000000,0.000000,0.0,0.0,0.0
75%,88.000000,30.000000,4.000000,0.000000,0.0,0.0,0.0
max,904.000000,480.000000,144.000000,128.000000,0.0,0.0,0.0


In [78]:
train_extra_desc = train_extra_desc[['hba1', 'hba2', 'hbd', 'nF']]

In [79]:
train_extra_desc

,hba1,hba2,hbd,nF
0,45.0,23.0,6.0,0.0
1,106.0,35.0,0.0,11.0
2,52.0,14.0,2.0,0.0
3,55.0,23.0,1.0,0.0
4,66.0,26.0,16.0,0.0
...,...,...,...,...
68608,67.0,10.0,0.0,5.0
68609,83.0,28.0,1.0,0.0
68610,116.0,32.0,0.0,0.0
68611,204.0,60.0,0.0,0.0


**Note:** logP, MR, TPSA are all 0, so we drop them.

In [80]:
train_extra = pd.concat([train_df, train_extra_desc], axis=1)
train_extra

,MOFname,volume [A^3],weight [u],surface_area [m^2/g],void_fraction,void_volume [cm^3/g],functional_groups,metal_linker,organic_linker1,organic_linker2,...,_cell_angle_gamma,n_atoms,mol_avg_mass,charges,mol_avg_radius,atoms_volume,hba1,hba2,hbd,nF
0,mof_unit_1,1116.667429,875.240600,0.00,0.07899,0.0607,COOH-OEt,3,4,11,...,1.575368,75,11.669907,92,59.653333,85.488522,45.0,23.0,6.0,0.0
1,mof_unit_2,2769.503842,2211.697211,603.61,0.13794,0.1040,F-OMe,10,44,57,...,1.691961,194,11.400559,250,60.515464,318.194213,106.0,35.0,0.0,11.0
2,mof_unit_3,1089.818728,773.687960,788.50,0.14874,0.1262,OMe-COOH,2,22,24,...,1.577559,82,9.435293,120,59.585366,92.531908,52.0,14.0,2.0,0.0
3,mof_unit_4,2205.198301,1304.638720,1441.53,0.21814,0.2220,H-SO3H,9,17,24,...,1.580099,112,11.648598,204,63.500000,185.375000,55.0,23.0,1.0,0.0
4,mof_unit_5,1137.800963,901.736120,0.00,0.07778,0.0591,NHMe-OH,2,1,22,...,1.595312,94,9.593000,90,58.425532,98.775910,66.0,26.0,16.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68608,mof_unit_68609,1188.302573,1001.700216,0.00,0.00000,0.0000,Pr-F,3,4,24,...,1.583947,119,8.417773,216,59.277311,124.277288,67.0,10.0,0.0,5.0
68609,mof_unit_68610,1506.660363,1493.296496,0.00,0.01108,0.0000,SO3H,10,42,46,...,1.700483,126,11.851548,126,62.460317,261.730396,83.0,28.0,1.0,0.0
68610,mof_unit_68611,2035.532738,1959.518320,0.00,0.00000,0.0000,OPr,4,14,22,...,1.034849,204,9.605572,366,61.578431,289.412179,116.0,32.0,0.0,0.0
68611,mof_unit_68612,3985.426053,3638.677280,0.00,0.00000,0.0000,OPr-Me,4,4,15,...,1.569257,364,9.996454,652,62.054945,541.832410,204.0,60.0,0.0,0.0


In [81]:
train_extra.to_csv('train_extra_3.csv', index=False)

## Test

In [82]:
test_df = pd.read_csv('test_extra_2.csv')
test_df

,MOFname,volume [A^3],weight [u],surface_area [m^2/g],void_fraction,void_volume [cm^3/g],functional_groups,metal_linker,organic_linker1,organic_linker2,...,_cell_length_b,_cell_length_c,_cell_angle_alpha,_cell_angle_beta,_cell_angle_gamma,n_atoms,mol_avg_mass,charges,mol_avg_radius,atoms_volume
0,mof_unit_68614,1208.301332,797.70936,586.54,0.11392,0.1039,OEt,2,5,26,...,9.084327,17.070225,1.556313,1.511002,1.575256,84,9.496619,128,59.761905,95.052588
1,mof_unit_68615,4126.414623,3733.65779,852.49,0.21367,0.1422,H-I,4,6,17,...,13.273001,27.655520,1.656643,1.574169,2.081566,158,23.630859,314,70.329114,349.189783
2,mof_unit_68616,1602.148373,747.21048,3155.73,0.33883,0.4375,CN-OH,3,11,17,...,10.697065,10.019137,1.569684,1.574747,1.571707,60,12.453633,94,62.433333,78.343492
3,mof_unit_68617,2436.629312,995.80232,3521.09,0.40464,0.5963,OMe,2,1,28,...,9.587345,23.461267,1.565566,1.590575,1.570093,94,10.593745,160,60.787234,108.832173
4,mof_unit_68618,3123.418006,1337.53800,2678.46,0.38959,0.5479,NO2-Pr,3,8,19,...,16.343084,16.653784,1.517304,1.599094,1.587196,118,11.335136,94,58.711864,120.398097
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16995,mof_unit_85609,32660.944605,4723.68288,5720.14,0.77614,3.2318,OH-NO2,3,3,14,...,38.924145,21.601534,1.565134,1.055786,1.568475,416,11.355106,768,61.711538,517.114417
16996,mof_unit_85610,5070.998617,1499.13262,4017.28,0.59192,1.2058,Me-OMe,3,1,11,...,18.775583,18.721517,1.905919,1.908772,1.910598,124,12.089839,196,62.201613,175.008856
16997,mof_unit_85611,4669.804446,1322.04892,4288.76,0.54950,1.1689,Me-CN,2,7,23,...,16.514905,15.943922,1.570403,1.586349,1.570638,112,11.804214,198,62.821429,135.867491
16998,mof_unit_85612,4682.120862,1213.51148,4331.86,0.60643,1.4091,OH-HCO,3,7,25,...,17.587298,16.887609,1.571806,1.571627,1.605246,104,11.668462,184,60.884615,118.374717


In [83]:
base_dir = 'mof_cif_test'

In [84]:
test_cif_desc = []
for i in tqdm(range(68613, 68613+len(test_df))):
    hba1, hba2, hbd, nF, logP, MR, TPSA = get_desc_cif(os.path.sep.join([base_dir,f'mof_unit_{i+1}.cif']))
    test_cif_desc.append([hba1, hba2, hbd, nF, logP, MR, TPSA])

100%|████████████████████████████████████████████████████████████████████████████| 17000/17000 [09:10<00:00, 30.87it/s]


In [85]:
test_extra_desc = pd.DataFrame(test_cif_desc, columns=['hba1', 'hba2', 'hbd', 'nF', 'logP', 'MR', 'TPSA'])

In [86]:
test_extra_desc

,hba1,hba2,hbd,nF,logP,MR,TPSA
0,52.0,14.0,0.0,0.0,0.0,0.0,0.0
1,53.0,26.0,0.0,0.0,0.0,0.0,0.0
2,30.0,16.0,4.0,0.0,0.0,0.0,0.0
3,50.0,20.0,0.0,0.0,0.0,0.0,0.0
4,66.0,10.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
16995,200.0,60.0,12.0,0.0,0.0,0.0,0.0
16996,71.0,27.0,0.0,0.0,0.0,0.0,0.0
16997,48.0,28.0,0.0,0.0,0.0,0.0,0.0
16998,50.0,26.0,4.0,0.0,0.0,0.0,0.0


In [87]:
# drop columns with all 0 values
test_extra_desc = test_extra_desc[['hba1', 'hba2', 'hbd', 'nF']]

In [88]:
test_extra = pd.concat([test_df, test_extra_desc], axis=1)
test_extra

,MOFname,volume [A^3],weight [u],surface_area [m^2/g],void_fraction,void_volume [cm^3/g],functional_groups,metal_linker,organic_linker1,organic_linker2,...,_cell_angle_gamma,n_atoms,mol_avg_mass,charges,mol_avg_radius,atoms_volume,hba1,hba2,hbd,nF
0,mof_unit_68614,1208.301332,797.70936,586.54,0.11392,0.1039,OEt,2,5,26,...,1.575256,84,9.496619,128,59.761905,95.052588,52.0,14.0,0.0,0.0
1,mof_unit_68615,4126.414623,3733.65779,852.49,0.21367,0.1422,H-I,4,6,17,...,2.081566,158,23.630859,314,70.329114,349.189783,53.0,26.0,0.0,0.0
2,mof_unit_68616,1602.148373,747.21048,3155.73,0.33883,0.4375,CN-OH,3,11,17,...,1.571707,60,12.453633,94,62.433333,78.343492,30.0,16.0,4.0,0.0
3,mof_unit_68617,2436.629312,995.80232,3521.09,0.40464,0.5963,OMe,2,1,28,...,1.570093,94,10.593745,160,60.787234,108.832173,50.0,20.0,0.0,0.0
4,mof_unit_68618,3123.418006,1337.53800,2678.46,0.38959,0.5479,NO2-Pr,3,8,19,...,1.587196,118,11.335136,94,58.711864,120.398097,66.0,10.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16995,mof_unit_85609,32660.944605,4723.68288,5720.14,0.77614,3.2318,OH-NO2,3,3,14,...,1.568475,416,11.355106,768,61.711538,517.114417,200.0,60.0,12.0,0.0
16996,mof_unit_85610,5070.998617,1499.13262,4017.28,0.59192,1.2058,Me-OMe,3,1,11,...,1.910598,124,12.089839,196,62.201613,175.008856,71.0,27.0,0.0,0.0
16997,mof_unit_85611,4669.804446,1322.04892,4288.76,0.54950,1.1689,Me-CN,2,7,23,...,1.570638,112,11.804214,198,62.821429,135.867491,48.0,28.0,0.0,0.0
16998,mof_unit_85612,4682.120862,1213.51148,4331.86,0.60643,1.4091,OH-HCO,3,7,25,...,1.605246,104,11.668462,184,60.884615,118.374717,50.0,26.0,4.0,0.0


In [89]:
test_extra.to_csv('test_extra_3.csv', index=False)